# Boosting

## Importamos las bibiliotecas que utilizaremos a lo largo del notebook

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import KFold, StratifiedKFold

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
from preprocessing import prepararSetDeDatos
from preprocessing import arbolDeDecisionPreprocessing

## Importamos los datos y los procesamos

In [ ]:
X = pd.read_csv('Datasets/tp-2020-2c-train-cols2.csv')
y = pd.read_csv('Datasets/tp-2020-2c-train-cols1.csv')
X,y = prepararSetDeDatos(X,y)

In [ ]:
X_b = arbolDeDecisionPreprocessing(X)

## Funciones Auxiliares

In [ ]:
def obtenerMejoresParametros(datosPreprocesados):
    mejor_valor = 0
    mejor_fc_perdida = None
    mejor_lr = None
    mejor_cantidad_estimadores = None
    mejor_criterio = None
    y_array=np.array(y)
    for fc_perdida in ["deviance", "exponential"]:
        for lr in [0.1, 0.01, 0.001]:
            for cantidad_estimadores in [10, 50, 100, 150]:
                for criterio in ["friedman_mse", "mse", "mae"]:
                    kf = StratifiedKFold(n_splits=5)
                    metricas = []
                    for fold_idx, (train_index, test_index) in enumerate(kf.split(datosPreprocesados, y_array)):
                        b = GradientBoostingClassifier(criterion=criterio, loss=fc_perdida, n_estimators=cantidad_estimadores, learning_rate=lr)
                        b.fit(datosPreprocesados[train_index], y_array[train_index].ravel())
                        predicciones = b.predict(datosPreprocesados[test_index])
                        score_obtenida = roc_auc_score(y_array[test_index],predicciones)
                        metricas.append(score_obtenida)

                    if np.mean(metricas) >= mejor_valor:
                        mejor_valor = np.mean(metricas)
                        mejor_fc_perdida = fc_perdida
                        mejor_lr = lr
                        mejor_cantidad_estimadores = cantidad_estimadores
                        mejor_criterio = criterio
            
    return mejor_valor, mejor_fc_perdida, mejor_lr, mejor_cantidad_estimadores, mejor_criterio

### Dividimos el set de datos en sets de training y test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_b, y, test_size=0.25, random_state=0)

### Buscamos los mejores parametros

In [ ]:
mejor_valor, mejor_fc_perdida, mejor_lr, mejor_cantidad_estimadores, mejor_criterio = obtenerMejoresParametros(X_b)

In [ ]:
print(f"El mejor valor fue de AUC fue: {mejor_valor}")
print(f"La mejor funcion de perdida encontrada que maximiza el AUC fue: {mejor_fc_perdida}")
print(f"El mejor learning rate encontrado que maximiza el AUC fue: {mejor_lr}")
print(f"La mejor cantidad de estimadores que maximiza el AUC fue: {mejor_cantidad_estimadores}")
print(f"El mejor criterio encontrado que maximiza el AUC fue: {mejor_criterio}")

### Evaluamos las metricas

In [ ]:
b = GradientBoostingClassifier(loss=mejor_fc_perdida, learning_rate=mejor_lr, n_estimators=mejor_cantidad_estimadores, criterion=mejor_criterio)
b.fit(X_train, y_train)
y_pred = b.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['No vuelve','Vuelve']))

### Matriz de confusion

In [ ]:
fig, ax = plt.subplots(dpi =150)   
sns.heatmap(confusion_matrix(y_test, y_pred), annot = True, vmin = 0, yticklabels=["Volveria","No volveria"], xticklabels=["Volveria", "No Volveria"], ax=ax)
ax.set_title("Matriz de confusión de Boosting")
ax.set_xlabel("Predicho")
ax.set_ylabel("Real")

### Graficamos la curva ROC

In [ ]:
fpr_b_test,tpr_b_test,thresholds_b_test = roc_curve(y_test,b.predict_proba(X_test)[:,1])

zero_test = np.argmin(np.abs(thresholds_b_test))

plt.plot(fpr_b_test,tpr_b_test,label="ROC Curve Boosting Test")
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.plot(fpr_b_test[zero_test],tpr_b_test[zero_test],'o',markersize=10,label="threshold zero test",fillstyle="none",c="k",mew=2)

plt.legend(loc=4)
plt.show()

In [ ]:
auc_b = roc_auc_score(y_test,b.predict_proba(X_test)[:, 1])
print("AUC para Boosting: {:.3f}".format(auc_b))